In [0]:
%sql
CREATE VOLUME IF NOT EXISTS practice.testschema.stream_input;
CREATE VOLUME IF NOT EXISTS practice.testschema.stream_output;
CREATE VOLUME IF NOT EXISTS practice.testschema.checkpoints;


In [0]:
with open("/Volumes/practice/testschema/stream_input/input1.json", "w") as f:
    f.write('{"id": 1, "name": "Alice", "value": 10}')
with open("/Volumes/practice/testschema/stream_input/input2.json", "w") as f:
    f.write('{"id": 2, "name": "Bob", "value": 15}')
with open("/Volumes/practice/testschema/stream_input/input3.json", "w") as f:
    f.write('{"id": 3, "name": "Alice", "value": 20}')


In [0]:
%sql
SHOW GRANTS ON VOLUME practice.testschema.stream_output;


In [0]:
%sql
SELECT current_user()


In [0]:
from pyspark.sql.types import StructType, StringType, IntegerType
checkpoint_path = "/Volumes/practice/testschema/checkpoints/my_stream_job"

schema = StructType() \
    .add("id", IntegerType()) \
    .add("name", StringType()) \
    .add("value", IntegerType())

df_stream = spark.readStream \
    .schema(schema) \
    .option("maxFilesPerTrigger", 1) \
    .json("/Volumes/practice/testschema/stream_input/")

df_stream.createOrReplaceTempView("streaming_data")

In [0]:
query = df_stream.writeStream \
    .format("delta") \
    .trigger(availableNow=True)\
    .option("checkpointLocation", checkpoint_path) \
    .start("/Volumes/practice/testschema/stream_output")

In [0]:
%sql
select * from delta.`/Volumes/practice/testschema/stream_output`